<div style="text-align: center; margin: 50px">

<h3>Building Boolean function oracles</h3>
</div>

In [ ]:
# Import qiskit and call it q for short so we can construct,simulate, and run quantum circuits!
import qiskit as q
# Import Numpy library and call it np so we can do maths!
import numpy as np
# Import the python plotting module of matplotlib and call it plt so we can draw graphs!
import matplotlib.pyplot as plt
# Tell matplotlib that we are in an Ipython notebook
%matplotlib inline

## Boolean function formula:

Consider a 3 variable Boolean function given as an Algebraic Normal form:

$$F(x_0,x_1,x_2) = x_0x_1x_2 + x_1x_2 + x_0$$

First one needs to determine the number of qubits required for constucting this oracle. This is determined by looking at the highest degree term.

In this case, this is a cubic function. This implies that the maximum number of qubits needed for constructing the bit oracle is 5: 3 for the input variables, 1 for the output variable and 1 internal qubit for the cubic term. 

### Variable assignment

The following qubits are assigned to the correponding Boolean variables:

$$ x_0 \rightarrow q_0 \\
   x_1 \rightarrow q_1 \\
   x_2 \rightarrow q_2 $$
   
   $q_3$ is the internal qubit, $q_4$ will store the final answer.

In [ ]:
# Therefore, this circuit needs 5 quantum bits but only 4 classical bits.
circ = q.QuantumCircuit(5,4)
display(circ.draw(output="mpl"))

## Input initialization

### *The input being passed to the Boolean function oracle can be implemented here:*

For purposes of this example Hadamard gates are applied to the input qubits $q_0,q_1,q_2$

$$ \begin{align} 
H \otimes H \otimes H \, |000\rangle & \rightarrow \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right) \otimes \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right) \otimes \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right) \\
& = \frac{1}{2\sqrt{2}} \left( |000\rangle + |001\rangle + |010\rangle + |011\rangle + |100\rangle + |101\rangle + |110\rangle + |111\rangle \right)
\end{align}$$

This will ensure that every possible input is simultaneously sent to the oracle in equal superposition.

 *The gates required to pass a particular input is commented out in the same block*

In [ ]:
# Input initializaion, use the inputs as necessary

# Use the three lines below to pass a single value:
#circ.x(0) # Use to set the x_0 to 1
#circ.x(1) # Use to set the x_1 to 1
#circ.x(2) # Use to set the x_2 to 1

# Use the three lines below to pass all possible inouts:
circ.h(0)
circ.h(1)
circ.h(2)


circ.barrier()
display(circ.draw(output="mpl"))

## Constructing the degree $3$ term

### *The Toffoli gate* 

The cubic term can be visulaised by AND and operations

$$ (x_0 \wedge x_1) \wedge x_2 $$

Each AND operation requires two Toffoli gates, the result of the first AND operation will be stored on the internal qubit ($q_3$) and this result is used as an input for the second AND operation, this reqs anther Toffoli gate. 



In [ ]:
# Constructing the cubic term
circ.ccx(0,1,3) # First AND operation
circ.ccx(2,3,4) # Second AND operation
circ.ccx(0,1,3) # Resetting the temporary qubit
circ.barrier()
display(circ.draw(output="mpl"))

## Constructing the degree $2$ term


The quadratic term can be visualised with a single toffoli gate

$$ x_1 \wedge x_2 $$

This result can be directly implemented on the ultimate target qubit $q_4$ 


In [ ]:
# Constructing the quadratic term
circ.ccx(1,2,4) # Single AND operation
circ.barrier()
display(circ.draw(output="mpl"))

## Constructing the degree $1$ term


The final term can be added (XORed) to the result through a single CNOT gate

In [ ]:
# Constructing the linear term
circ.cx(0,4) # CNOT gate
circ.barrier()
display(circ.draw(output="mpl"))

## Measuring the output


Finally, measurement operations are applied with the following bits representing the outcomes.

$$ q_0 \rightarrow c_3 \\
   q_1 \rightarrow c_2 \\
   q_2 \rightarrow c_1 \\
   q_4 \rightarrow c_0 $$
   
This ordering is done here to make sure we can read off the input and output values according to predefined conventions.

In [ ]:
# Performing measurments
circ.measure([0,1,2,4],[3,2,1,0])
display(circ.draw(output="mpl"))

## Testing the Circuit on the QASM simulator


The following commands are used to implement the circuit on the simulator.

In [ ]:
backend = q.Aer.get_backend('qasm_simulator')  # specifying that we will use qasm simulator
job = q.execute(circ, backend, shots = 1024)  # shots=1024 specifies that the circuit will be run 1024 times
result = job.result()
# getting the counts, i.e., the fraction of times the circuit gave all the possible results
counts = result.get_counts(circ)  
graph = q.visualization.plot_histogram(counts)
display(graph)

# Do the measurments on a real quantum computer

In [ ]:
from qiskit import IBMQ

# Load local account information
IBMQ.save_account('paste API token here', overwrite=True) 
provider = IBMQ.load_account()

In [ ]:

from qiskit.providers.ibmq import least_busy
shots = 256

#IBMQ.load_account()
# Get the least busy backend
provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 
                                       and not x.configuration().simulator 
                                       and x.status().operational==True))
print("least busy backend: ", backend)


# Run our circuit
job = q.execute(circ, backend=backend, shots=shots)

In [ ]:
job.status()

In [ ]:
# Monitoring our job
from qiskit.tools.monitor import job_monitor
job_monitor(job)

In [ ]:
job.job_id()



In [ ]:
# Plotting our result
result = job.result()
q.visualization.plot_histogram(result.get_counts(circ))